In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option("display.max_columns", 50)

In [ ]:
os.chdir('data/')

In [ ]:
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk('.'):
    for file in f:
        #if '.txt' in file:
            #files.append(os.path.join(r, file))
        files.append(os.path.join(r, file))
for f in files:
    print(f)

In [ ]:
data_dict=pd.read_excel('./Data_Dictionary.xlsx')
data_dict

## Datasets

In [ ]:
train=pd.read_csv('./train_clubmahindra.csv')
test=pd.read_csv('./test_clubmahindra.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
plt.figure(figsize=(12.0,10))
sns.heatmap(train.corr(),annot=True,linewidths=0.3)
plt.show()

In [ ]:
train.info()

In [ ]:
import missingno
missingno.matrix(train)

In [ ]:
missingno.matrix(test)

## Reservation ID

In [ ]:
#test=test.drop(['reservation_id'],axis=1)
train=train.drop(['reservation_id'],axis=1)

## Booking Date,Checkin Date,Checkout_date

In [ ]:
train['booking_date']=pd.to_datetime(train['booking_date'],format='%d/%m/%y')
test['booking_date']=pd.to_datetime(test['booking_date'],format='%d/%m/%y')

In [ ]:
train['booking_date']

In [ ]:
train['checkin_date']=pd.to_datetime(train['checkin_date'],format='%d/%m/%y')
test['checkin_date']=pd.to_datetime(test['checkin_date'],format='%d/%m/%y')

In [ ]:
train['checkin_date']

In [ ]:
train['checkout_date']=pd.to_datetime(train['checkout_date'],format='%d/%m/%y')
test['checkout_date']=pd.to_datetime(test['checkout_date'],format='%d/%m/%y')

In [ ]:
train['days_stayed']=train['checkout_date']-train['checkin_date']
train['days_stayed']=train['days_stayed']/np.timedelta64(1,'D')
test['days_stayed']=test['checkout_date']-test['checkin_date']
test['days_stayed']=test['days_stayed']/np.timedelta64(1,'D')

In [ ]:
train['days_stayed'].min()

In [ ]:
train['booking_to_checkin']=(train['checkin_date']-train['booking_date'])/np.timedelta64(1,'D')
test['booking_to_checkin']=(test['checkin_date']-test['booking_date'])/np.timedelta64(1,'D')

In [ ]:
train[train['booking_to_checkin']<0]

In [ ]:
print(test[test['booking_to_checkin']<0].index)

In [ ]:
train[train['booking_to_checkin']<0].index

In [ ]:
test[test['booking_to_checkin']<0].index.values

In [ ]:
train.loc[train[train['booking_to_checkin']<0].index.values,['booking_date','checkin_date']]

In [ ]:
train['checkin_date']

In [ ]:
train.loc[train[train['booking_to_checkin']<0].index.values,['booking_date']]=train['checkin_date']
#test.loc[test[test['booking_to_checkin']<0].index.values,['booking_date']]=test['checkin_date']

In [ ]:
test.loc[test[test['booking_to_checkin']<0].index.values,['booking_date']]=test['checkin_date']

In [ ]:
train.head()

In [ ]:
train.loc[[1,15, 34, 60, 79, 81, 113, 116, 129, 130, 147, 149, 155],['booking_date']]

In [ ]:
train['booking_date']

In [ ]:
train.loc[[15, 34, 60, 79, 81, 113, 116, 129, 130, 147, 149, 155],['booking_to_checkin']]=(train['checkin_date']-train['booking_date'])/np.timedelta64(1,'D')

In [ ]:
test.loc[[41, 57],['booking_to_checkin']]=(test['checkin_date']-test['booking_date'])/np.timedelta64(1,'D')

In [ ]:
train['booking_to_checkin'].value_counts(bins=5)

In [ ]:
train['booking_to_checkin'].min()

In [ ]:
#train['month_of_checkin'] = pd.to_datetime(train['checkin_date']).dt.to_period('M')
train['month_of_checkin'] = pd.DatetimeIndex(train['checkin_date']).month
print(train['month_of_checkin'].head())
test['month_of_checkin'] = pd.DatetimeIndex(test['checkin_date']).month
print(test['month_of_checkin'].head())

In [ ]:
train=train.drop(['checkin_date','checkout_date','booking_date'],axis=1)
test=test.drop(['checkin_date','checkout_date','booking_date'],axis=1)

In [ ]:
train.head()

## Channel_Code, Main Product Code

In [ ]:
train['channel_code'].value_counts(dropna=False)

In [ ]:
sns.boxplot(data=train,x='channel_code',y='amount_spent_per_room_night_scaled')

In [ ]:
train_cc_1=train[train['channel_code']==1]
train_cc_2=train[train['channel_code']==2]
train_cc_3=train[train['channel_code']==3]

In [ ]:
from scipy.stats import f_oneway

In [ ]:
f_oneway(train_cc_1['amount_spent_per_room_night_scaled'],train_cc_2['amount_spent_per_room_night_scaled'],
        train_cc_3['amount_spent_per_room_night_scaled'])

In [ ]:
train['main_product_code'].value_counts(dropna=False)

In [ ]:
train_mpc = train.groupby('main_product_code')
train_mpc_1=train_mpc.get_group(1)
train_mpc_2=train_mpc.get_group(2)
train_mpc_3=train_mpc.get_group(3)
train_mpc_4=train_mpc.get_group(4)
train_mpc_7=train_mpc.get_group(7)

In [ ]:
f_oneway(train_mpc_1['amount_spent_per_room_night_scaled'],train_mpc_2['amount_spent_per_room_night_scaled'],train_mpc_3['amount_spent_per_room_night_scaled'],
        train_mpc_4['amount_spent_per_room_night_scaled'],train_mpc_7['amount_spent_per_room_night_scaled'])

## numberofadults, numberofchildren

In [ ]:
train['numberofadults'].value_counts()

In [ ]:
train['numberofchildren'].value_counts()

In [ ]:
train[train['numberofchildren']>10]

In [ ]:
#train['total_customers']=train['numberofadults']+train['numberofchildren']#
#test['total_customers']=test['numberofadults']+test['numberofchildren']

In [ ]:
#train['total_customers'].value_counts(bins=5)

## persontravellingid

In [ ]:
train['persontravellingid'].value_counts(dropna=False)

## resort_region_code

In [ ]:
train['resort_region_code'].value_counts(dropna=False)

## resort_type_code

In [ ]:
train['resort_type_code'].value_counts(dropna=False)

## room_type_booked_code

In [ ]:
train['room_type_booked_code'].value_counts(dropna=False)

## roomnights

In [ ]:
train['roomnights'].value_counts(dropna=False,bins=5)

## season_holidayed_code

In [ ]:
train['season_holidayed_code'].value_counts(dropna=False)

In [ ]:
train[train['season_holidayed_code'].isnull()]

In [ ]:
train['season_holidayed_code']=train['season_holidayed_code'].fillna(method='ffill')
test['season_holidayed_code']=test['season_holidayed_code'].fillna(method='ffill')

## state_code_residence

In [ ]:
train['state_code_residence'].value_counts(dropna=False)

In [ ]:
train['state_code_residence']=train['state_code_residence'].fillna(method='bfill')
test['state_code_residence']=test['state_code_residence'].fillna(method='bfill')

## state_code_resort

In [ ]:
train['state_code_resort'].value_counts()

## total_pax

In [ ]:
train['total_pax'].value_counts()

## member_age_buckets

In [ ]:
train['member_age_buckets'].value_counts()

## booking_type_code

In [ ]:
train['booking_type_code'].value_counts()

## memberid

In [ ]:
train['memberid'].value_counts()

In [ ]:
train=train.drop(['memberid'],axis=1)
test=test.drop(['memberid'],axis=1)

## cluster_code

In [ ]:
train['cluster_code'].value_counts()

## reservationstatuscode

In [ ]:
train['reservationstatusid_code'].value_counts()

## resortid

In [ ]:
train['resort_id'].value_counts()

## amount_spent_per_room_night_scaled

In [ ]:
train['amount_spent_per_room_night_scaled']

In [ ]:
sns.distplot(train['amount_spent_per_room_night_scaled'])

## Creating Dummies

In [ ]:
df=pd.concat([train,test],axis=0,sort=False)

In [ ]:
print('Number of rows in the dataset',df.shape[0])
print('Number of columns in the dataset',df.shape[1])

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df.index

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le=LabelEncoder()
ohe=OneHotEncoder(sparse=False,categories='auto')

In [ ]:
df['persontravellingid']=le.fit_transform(df['persontravellingid'])
df['resort_id']=le.fit_transform(df['resort_id'])

In [ ]:
df.select_dtypes(include='object').drop(['reservation_id'],axis=1)

In [ ]:
ohe.fit_transform(df.select_dtypes(include='object').drop(['reservation_id'],axis=1))

In [ ]:
ohe.get_feature_names(df.select_dtypes(include='object').drop(['reservation_id'],axis=1).columns)

In [ ]:
df.select_dtypes(include='object').drop(['reservation_id'],axis=1)

In [ ]:
temp=pd.DataFrame(ohe.fit_transform(df.select_dtypes(include='object').drop(['reservation_id'],axis=1)),
                  columns=ohe.get_feature_names(df.select_dtypes(include='object').drop(['reservation_id'],axis=1).columns)
                 ,index=df.index)

In [ ]:
temp.head()

In [ ]:
df=df.drop(df.select_dtypes(include='object'),axis=1)

In [ ]:
df.shape

In [ ]:
temp.shape

In [ ]:
df_final=pd.concat([df,temp],axis=1)

In [ ]:
df_final.head()

In [ ]:
df_final=df_final.loc[:,~df_final.columns.duplicated()]

In [ ]:
train.shape[0]

In [ ]:
df_train=df_final.iloc[:train.shape[0],:]
df_test=df_final.iloc[train.shape[0]:,:]